In [ ]:
#! echo "/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/groqapi.txt"

In [ ]:
!pip install -r "/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/requirements.txt"

In [19]:
from google.colab import userdata
grq_api = userdata.get('grog_api')
serper_api = userdata.get('serper_api')

In [17]:
from groq import Groq
client = Groq(api_key=grq_api)

try:
    # Send prompt to Groq API
    completion = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": "are you free to use ? what is the usage limit for you?"}],
        temperature=0.7,
        max_completion_tokens=200,
        top_p=1,
        stream=True,
        stop=None,
    )

    # Stream output
    print("Response: ", end="")
    for chunk in completion:
        print(chunk.choices[0].delta.content or "", end="")

except Exception as e:
    print(f"\nError during API request: {e}")

Response: I'm a large language model, and my usage is governed by several factors. Here are some details:

1. **Fair use policy**: My usage is governed by a fair use policy, which means I can be used for personal, educational, or research purposes. However, I may not be used for commercial purposes without permission from my developers.
2. **Query limits**: I don't have a strict query limit, but I'm designed to handle a large volume of conversations. If I detect that you're making an excessive number of requests, I may slow down or limit my responses.
3. **Session limits**: I can maintain multiple conversations simultaneously, but each session has a limit on the number of turns (i.e., back-and-forth conversations). If we reach this limit, I may ask you to start a new conversation or take a break.
4. **Content limits**: I can process and respond to a wide range of content, including text, images, and other media. However

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# # Load environment variables
# from helper import load_env
# load_env()

import os
import yaml
from crewai import Agent, Task, Crew

In [2]:
from pydantic import BaseModel, Field
from typing import List, Optional

class MalwareSample(BaseModel):
    hash: str = Field(..., description="The hash of the malware sample (e.g., SHA-256, MD5).")
    source: str = Field(..., description="The source from where the malware intelligence was gathered.")
    classification: Optional[str] = Field(None, description="The classification of the malware (packer, dropper, loader).")
    description: Optional[str] = Field(None, description="Brief description of the malware behavior.")
    threat_level: Optional[str] = Field(None, description="Severity of the malware (Low, Medium, High).")

class OSINTReport(BaseModel):
    collected_samples: List[MalwareSample] = Field(..., description="A list of collected malware samples with metadata.")
    intelligence_sources: List[str] = Field(..., description="Sources used for gathering information.")

class ThreatAnalysis(BaseModel):
    indicators_of_compromise: List[str] = Field(..., description="List of IOCs such as IPs, domains, file hashes, and URLs.")
    behavior_analysis: str = Field(..., description="Detailed behavior analysis of the malware.")
    attack_techniques: List[str] = Field(..., description="MITRE ATT&CK techniques associated with the malware.")

class ClassificationReport(BaseModel):
    categorized_samples: List[MalwareSample] = Field(..., description="List of malware samples with classification details.")
    classification_summary: str = Field(..., description="Summary of classification criteria used.")

class ResearchSummary(BaseModel):
    key_findings: str = Field(..., description="High-level summary of all gathered intelligence and analysis.")
    recommended_next_steps: List[str] = Field(..., description="Suggested next steps for research and further analysis.")
    safe_analysis_guidelines: str = Field(..., description="Best practices for safely handling and analyzing the malware.")

class MalwareAnalysisOutput(BaseModel):
    osint_report: OSINTReport = Field(..., description="OSINT intelligence gathering report.")
    threat_analysis: ThreatAnalysis = Field(..., description="Technical analysis of the malware.")
    classification_report: ClassificationReport = Field(..., description="Categorization of the malware.")
    research_summary: ResearchSummary = Field(..., description="Final research findings and recommendations.")


In [ ]:
tool = WebsiteSearchTool(
    config=dict(
        llm=dict(
            provider="ollama", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="llama2",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="google", # or openai, ollama, ...
            config=dict(
                model="models/embedding-001",
                task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)

In [ ]:
!pip install 'litellm[proxy]'

In [20]:
import yaml
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
os.environ['GROQ_API_KEY'] = grq_api
os.environ['SERPER_API_KEY'] = serper_api

# Define Groq LLM model
groq_llm = "groq/llama-3.1-8b-instant"

# Define file paths for YAML configurations
files = {
    'agents': '/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/config/agents.yaml',
    'tasks': '/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

# Creating Agents
osint_collector_agent = Agent(
    config=agents_config['osint_collector_agent'],
    tools=[SerperDevTool()],
    llm=groq_llm,
)

threat_analyst_agent = Agent(
    config=agents_config['threat_analyst_agent'],
    tools=[SerperDevTool()],
    llm=groq_llm,
)

malware_categorizer_agent = Agent(
    config=agents_config['malware_categorizer_agent'],
    tools=[ScrapeWebsiteTool()],
    llm=groq_llm,
)

research_advisor_agent = Agent(
    config=agents_config['research_advisor_agent'],
    tools=[],
    llm=groq_llm,
)

# Creating Tasks
search_malware_intelligence_task = Task(
    config=tasks_config['search_malware_intelligence'],
    agent=osint_collector_agent
)

analyze_malware_data_task = Task(
    config=tasks_config['analyze_malware_data'],
    agent=threat_analyst_agent
)

classify_malware_type_task = Task(
    config=tasks_config['classify_malware_type'],
    agent=malware_categorizer_agent,
    context=[search_malware_intelligence_task, analyze_malware_data_task]
)

generate_research_summary_task = Task(
    config=tasks_config['generate_research_summary'],
    agent=research_advisor_agent,
    context=[search_malware_intelligence_task, analyze_malware_data_task, classify_malware_type_task],
    output_pydantic=MalwareAnalysisOutput
)

# Creating Crew
malware_analysis_crew = Crew(
    agents=[
        osint_collector_agent,
        threat_analyst_agent,
        malware_categorizer_agent,
        research_advisor_agent
    ],
    tasks=[
        search_malware_intelligence_task,
        analyze_malware_data_task,
        classify_malware_type_task,
        generate_research_summary_task
    ],
    verbose=True
)

# Run the crew
if __name__ == "__main__":
    malware_analysis_crew.kickoff()


# Agent: OSINT Researcher
## Task: Conduct an extensive OSINT search for information on malware families,  focusing on packers, droppers, and loaders. Gather data from sources like  VirusTotal, Hybrid Analysis, MITRE ATT&CK, and cybersecurity research forums.

 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation error for SerperDevToolSchema
search_query
  Input should be a valid string [type=string_type, input_value={'description': 'packers ...ers loaders VirusTotal'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type.
 Tool Search the internet with Serper accepts these inputs: Tool Name: Search the internet with Serper
Tool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the internet', 'type': 'str'}}
Tool Description: A tool that can be used to search the internet with a search_query. Supports different search types: 'search' 

ERROR:root:LiteLLM call failed: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01jnmhpvvbenwrkhevabm1d2g0` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 4922, Requested 2967. Please try again in 18.881s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}





LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

 Error during LLM call: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01jnmhpvvbenwrkhevabm1d2g0` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 4922, Requested 2967. Please try again in 18.881s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}



RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01jnmhpvvbenwrkhevabm1d2g0` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 4922, Requested 2967. Please try again in 18.881s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}


In [8]:
import yaml
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool


# Define Groq LLM model
groq_llm = "llama-3.1-8b-instant"

# Define file paths for YAML configurations
files = {
    'agents': '/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/config/agents.yaml',
    'tasks': '/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

# Creating Agents
osint_collector_agent = Agent(
    config=agents_config['osint_collector_agent'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=groq_llm,
)

threat_analyst_agent = Agent(
    config=agents_config['threat_analyst_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool(
            adapter=EmbedchainAdapter(config=AppConfig(api_key=grq_api))  # Providing Groq API key to EmbedchainAdapter
        )],
    llm=groq_llm,
)

malware_categorizer_agent = Agent(
    config=agents_config['malware_categorizer_agent'],
    tools=[WebsiteSearchTool(), ScrapeWebsiteTool()],
    llm=groq_llm,
)

research_advisor_agent = Agent(
    config=agents_config['research_advisor_agent'],
    tools=[],
    llm=groq_llm,
)

# Creating Tasks
search_malware_intelligence_task = Task(
    config=tasks_config['search_malware_intelligence'],
    agent=osint_collector_agent
)

analyze_malware_data_task = Task(
    config=tasks_config['analyze_malware_data'],
    agent=threat_analyst_agent
)

classify_malware_type_task = Task(
    config=tasks_config['classify_malware_type'],
    agent=malware_categorizer_agent,
    context=[search_malware_intelligence_task, analyze_malware_data_task]
)

generate_research_summary_task = Task(
    config=tasks_config['generate_research_summary'],
    agent=research_advisor_agent,
    context=[search_malware_intelligence_task, analyze_malware_data_task, classify_malware_type_task],
    output_pydantic=MalwareAnalysisOutput
)

# Creating Crew
malware_analysis_crew = Crew(
    agents=[
        osint_collector_agent,
        threat_analyst_agent,
        malware_categorizer_agent,
        research_advisor_agent
    ],
    tasks=[
        search_malware_intelligence_task,
        analyze_malware_data_task,
        classify_malware_type_task,
        generate_research_summary_task
    ],
    verbose=True
)

# Run the crew
if __name__ == "__main__":
    malware_analysis_crew.kickoff()


ImportError: cannot import name 'EmbedchainAdapter' from 'crewai_tools' (/usr/local/lib/python3.11/dist-packages/crewai_tools/__init__.py)

In [ ]:
import os
from crewai_tools import WebsiteSearchTool, EmbedchainAdapter
from embedchain.config import AppConfig

# ... (rest of your imports and code) ...


threat_analyst_agent = Agent(
    config=agents_config['threat_analyst_agent'],
    tools=[
        SerperDevTool(),
        WebsiteSearchTool(
            adapter=EmbedchainAdapter(config=AppConfig(api_key=os.environ.get("GROQ_API_KEY")))  # Providing Groq API key to EmbedchainAdapter
        )
    ],
    llm=groq_llm,
)

malware_categorizer_agent = Agent(
    config=agents_config['malware_categorizer_agent'],
    tools=[
        WebsiteSearchTool(
            adapter=EmbedchainAdapter(config=AppConfig(api_key=os.environ.get("GROQ_API_KEY")))  # Providing Groq API key to EmbedchainAdapter
        ),
        ScrapeWebsiteTool()
    ],
    llm=groq_llm,
)

# ... (rest of your code) ...